# Pretrained Networks

![CNN-Architecture-over-a-timeline](https://www.aismartz.com//blog/wp-content/uploads/2019/10/CNN-Architecture-over-a-timeline.jpg)

https://www.aismartz.com/blog/cnn-architectures/

+ WHAT?
    + it is a network which was trained on a large dataset on a large-scale-image classification Task. One can usit as it is for image classification or for transfer learning so we can custumise the model for a new task
    
+ BENEFITS
    + we don't have to train the model
    + very easy to incorporate 
    + fast simulation
    + we can achieve very good performaces
    
+ EXAMPLE
 + U net: recognize feature on medical images
 + MobileNet
 + VGG16/19
 + ResNet
 + InceptionV3
 
 Most of the pretrained networks available in keras are trained on [1000 different classes](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a) that were used from the [Imagenet](https://www.image-net.org/update-mar-11-2021.php) dataset

In [ ]:
# import libraries
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential,Input,Model

import numpy as np
import matplotlib.pyplot as plt

#### Lets load an image from internet that belong under any of the 1000 classes (few images present in images directory )

We will then use a pretrained model (ResNet50) and check the classification result

#### Information about the other CNN architectures
https://www.tensorflow.org/api_docs/python/tf/keras/applications


https://keras.io/api/applications/

### Read the image as a PIL object and resize to a desired dimension(ideally the same shape as the one which the pretrained model is trained on)

In [ ]:
img = image.load_img('images/download.jpeg',target_size=(224,224)) 

In [ ]:
type(img)

In [ ]:
#convert image to array, can also specify datatype
img = image.img_to_array(img,dtype='uint8')

In [ ]:
img.shape

In [ ]:
#plot image 
plt.imshow(img)

#### Load model
#### As an example we are using the ResNet50 pretrained model

In [ ]:
# initialize the model
model = ResNet50()

In [ ]:
#show model summary
model.summary()

Take a look at how many parameters needs to be trained for this model

In [ ]:
#check shape required by model
model.input.shape

##### Expand dimensions

In [ ]:
# Reshape to match the input shape required by the model
img = np.expand_dims(img,axis=0) # or img = img.reshape(1,224,224,3)

In [ ]:
img.shape

In [ ]:
pred = model.predict(img)

In [ ]:
#shape of pred
pred.shape

In [ ]:
# decode labels
decode_predictions(pred)

!! We got the whole result while predicting from our pretrained model !! 

This brings us to point of applying preprocessing to the image when using pretrained models

### Preprocess

## why do we need to preprocess image before loading to CNN model ??

To get best predictions we need to preprocess the images, just as it was done while the researchers were training them

For eg. in Resnet model
> The only preprocessing we do is subtracting the mean RGB value, computed on the training set, from each pixel.

https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/preprocess_input

In [ ]:
img = preprocess_input(img) # preprocess the image in the same method that is used for the pretrained model

In [ ]:
plt.imshow(img[0])

#### Predict

In [ ]:
pred = model.predict(img)

In [ ]:
#shape of pred
pred.shape

In [ ]:
# decode labels
decode_predictions(pred)

The model now predicts the image accurately after predicting using the preprocessed image

Now load the `object.png` image from the `imgage` directory and check the results using ResNet50

You should be getting the predicted result as `cleaver`

Discuss why the model predicted a `phone` as `cleaver`

# Transfer Learning

## Transfer Learning for Neural Networks

> Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem. For instance, features from a model that has learned to identify racoons may be useful to kick-start a model meant to identify tanukis (japanese racoons).

__The benefits of transfer learning are:__
* you can reuse pre-trained networks
* it saves lots of training time
* it allows you to train with very small training datasets

__Procedure__
1. Take the weights and architecture of a [pre-trained network](https://keras.io/api/applications/)
2. Load the "convolutional base" of the model (everything except the final dense layers)
3. Freeze all the layers of the base (weights become fixed)
4. Add a fully connected dense layer on top
5. **Add a task specific dense output layer**
6. Compile and fit the model to your data

<img src='https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Transfer-learning-base-model.jpg?resize=512%2C375&ssl=1'>

## ImageDataGenerator
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

- `class ImageDataGenerator`: Generate batches of tensor image data with **optional** real-time data augmentation

This is an efficient way to load data when you are working with lots of image data which does not fit into the memory (RAM) as you can load and train the model in small batches

In [ ]:
# folder names containing images of the things you want to classify
classes = ['phone','wallet']

Data augmentation can address a variety of challenges when training a CNN model, such as limited or imbalanced data, overfitting, and variation and complexity.  eg Flipping,Rotation, Translation, Scaling ... of images

This technique can increase the size of the dataset and balance the classes by applying different transformations.

In [ ]:
# define an image data generator
# Data augmentation: Applies random distortions and transformations to the images (only on your training data!).

data_gen = image.ImageDataGenerator(
    # define the preprocessing function that should be applied to all images
    preprocessing_function=preprocess_input,
    # fill_mode='nearest',
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True, 
    # zoom_range=0.2,
    # shear_range=0.2
)

In [ ]:
!unzip data.zip

In [ ]:
# a generator that returns batches of X and y arrays
train_data_gen = data_gen.flow_from_directory(
        directory='data/train',
        class_mode="categorical",
        classes=classes,
        batch_size=150,
        target_size=(224, 224),
)

In [ ]:
val_data_gen = data_gen.flow_from_directory(
        directory='data/validation/',
        class_mode="categorical",
        classes=classes,
        batch_size=150,
        target_size=(224, 224),
)

In [ ]:
train_data_gen.class_indices

In [ ]:
classes

## Create CNN Model

### 1. Select the convolutional base / Pretrained network

### 2. Freeze the weights

### 3. Add your own dense layers on top

In [ ]:
import tensorflow.keras.backend as K
K.clear_session()

#1. Select the convolutional base / Pretrained network
base_model = ResNet50(include_top=False)

In [ ]:
base_model.summary()

When loading a given model, the “include_top” argument can be set to False, in which case the fully-connected output layers of the model used to make predictions is not loaded, allowing a new output layer to be added and trained.

### 2. Freeze the weights

In [ ]:
#2. Freeze the weights in order to not retrain the loaded pre-trained model
base_model.trainable= False

In [ ]:
base_model.summary()

### 3. Add your own dense layers on top, in our case we have only 2 classes to classify `phone/wallet`

Instead of building CNN model from scratch, we are using ResNet50 and fine tuning it to fit to our dataset by editing the last few layers of our pretrained model

In [ ]:
# 3. Create your model with pretrained network as base model
inputs = Input(shape=(224,224,3))

base = base_model(inputs)

# can also add additional cnn layers if necessary

# dont forget to flatten out before the final layer
flatten = GlobalAveragePooling2D()(base)

outputs = Dense(2,activation='softmax')(flatten)

model_tf = Model(inputs,outputs)

In [ ]:
model_tf.summary()

### 4. Compile and train!

In [ ]:
model_tf.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_tf.fit(train_data_gen,
          verbose=2, # how the training log should get printed 
          epochs=10,
          validation_data=val_data_gen)

### (5. Use it to predict)

In [ ]:
model_tf.predict(img)

In [ ]:
plt.bar(x = classes, height = model_tf.predict(img)[0])

In [ ]:
# For remapping the index values of highest prediction probability to its respective class

pred = model_tf.predict(img)

preds_cls = list(train_data_gen.class_indices.keys())[list(train_data_gen.class_indices.values()).index(np.argmax(pred,axis=-1))]
preds_cls

### (6. Save your model for later)

In [ ]:
model.save('models/wallet_phone.h5')

## Exercise
- Try out different pretrained models as base
- Choose any [image classification dataset](https://www.kaggle.com/datasets?search=image) and build your CNN model (optional)
    - Try building your CNN model from scratch
    - Use transfer learning approach to train your CNN model

- For running big CNN models with bigger datasets, sometimes your laptop hardware may not be sufficient 
    - You can use free cloud resources for training such models as they provide you with free GPU as well (GPU speeds up training)
    - [Google Colab](https://colab.research.google.com/)
    - [Kaggle kernel](https://www.kaggle.com/code) (click on `New notebook`, also you will need to verify phone number inorder to use GPU)

# Loading the entire images and labels into arrays (alternate)

In [ ]:
# Let's explore the data folder
import os
base_path = 'data/train/'

# Let's define the classes
classes = os.listdir(base_path)

In [ ]:
 for class_ in classes:
        print(class_)

In [ ]:
def load_image(base_path):
    """it loads all the image into X and the classes in y """
    X_list = []
    y_list = []
    classes = os.listdir(base_path)
    for class_ in classes:
        if class_!='.DS_Store':
        
            files = os.listdir(base_path+class_)
            for file in files:
                pic = image.load_img(path=base_path+class_+'/'+f'{file}',target_size=(224,224))
                numpy_image = np.array(pic)
                processed_image = preprocess_input(numpy_image)
                X_list.append(processed_image)
                y_list.append(class_)

    X = np.array(X_list)
    y = np.array(y_list)
    
    return X, y, classes

In [ ]:
X,y,classes= load_image(base_path)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
my_dict = {"wallet":0, "phone":1}

In [ ]:
# map strings to binary labels
y = np.vectorize(my_dict.get)(y)
y

In [ ]:
!rm -rf __MACOSX
!rm -rf data/
!rm -rf .DS_Store

# Additional links


https://keras.io/guides/transfer_learning/

https://www.tensorflow.org/tutorials/images/data_augmentation



[Guide for Transfer learning](https://neptune.ai/blog/transfer-learning-guide-examples-for-images-and-text-in-keras)